# Loading human plays' data

In [ ]:
import json
import pandas as pd
import numpy as np
import mmap

In [ ]:
# games_jsons = []
# ix = 0
# with open("../data/diplomacy-v1-27k-msgs/standard_no_press.jsonl", "r") as json_file:
#     for line in json_file:
#         games_jsons.append(json.loads(line))

In [ ]:
# Load the json data
completed = False
games_jsons = []
with open("../data/diplomacy-v1-27k-msgs/standard_no_press.jsonl", "r+b") as json_file:
    with mmap.mmap(json_file.fileno(), length=0, access=mmap.ACCESS_READ) as mmap_object:
        for i, line in enumerate(iter(mmap_object.readline, b"")):
            tmp = json.loads(line.decode("utf-8"))
            if tmp["map"] == "standard":
                if completed:
                    for phase in tmp["phases"]:
                        if phase["name"] == "COMPLETED":
                            games_jsons.append(tmp)
                else:
                    games_jsons.append(tmp)
            if len(games_jsons) == 30:
                print("last game id is", i)
                break

## Uncomment to load RL single game data

In [ ]:

# games_jsons = []
# json_file_path = "../data/game_rl_0.json"

# with open(json_file_path, 'r') as j:
#       games_jsons.append(json.loads(j.read()))

In [ ]:
# Convert to a pandas dataframe
df = pd.DataFrame(games_jsons)

In [ ]:
# df = df.loc[df["map"] == "standard"].reset_index(drop=True)

In [ ]:
df


In [ ]:
games = df['phases']

In [ ]:
games.apply(lambda x: x[0])

In [ ]:
type(games)

In [ ]:
for game in games:
    for ix, iy in enumerate(game):
        game[ix]['phase_id'] = ix

In [ ]:
def flatten_json(input):
    out = {}
    out['game_id'] = input['state']['game_id']
    out['phase_id'] = input['phase_id']
    out['phase_name'] = input['name']
    results_units_keys = [x for x in input['results']]

    assert input['orders'].items() 
    for player, orders in input['orders'].items():
        if orders is not None:
            if orders:
                for order in orders:
                    out['coordinator'] = player
                    out['type'] = order.split()[0]
                    out['current_location'] = order.split()[1]
                    out['action'] = order.split()[2]
                    unit = order.split()[0]+ ' ' + order.split()[1]
                    # print("unit result is: ", unit , unit in results_units, results_units)
                    if input['results']:
                        out['results'] = input['results'][unit]
                    else:
                        print("empty results")
                        print(order)
                    if unit in results_units_keys:
                        results_units_keys.remove(unit)
                    out['impact_location'] = []
                    if out['action'] == '-' or out['action'] == 'R':
                        assert len(order.split()) == 4 or len(order.split()) == 5, order
                        out['impact_location'] = order.split()[3]
                    yield(out)
                
            else:
                # uncmomment these if you need a row for empty orders (in group by you will get 1 instead of 0)
                # out['coordinator'] = player
                # out['type'] = -1
                # out['current_location'] = -1
                # out['action'] = -1
                # out['results'] = -1
                # out['impact_location'] = -1
                # if out['action'] == '-' or out['action'] == 'R':
                #     assert len(order.split()) == 4 or len(order.split()) == 5, order
                #     out['impact_location'] = order.split()[3]
                # yield(out)
                pass

    if len(results_units_keys)>0:
        # uncomment for completed games
        all_orders = [order for order in input['orders'].values()]
        total_len = 0
        for l in all_orders:
            total_len += len(l)
        if total_len == 0:
            print(out)
        assert [order for order in input['orders'].values()]
        for unit in results_units_keys:
            if unit == 'WAIVE': ## FIXME: double check this later
                continue
            assert len(unit.split()) == 2, unit
            out['coordinator'] = 'RA'
            for player in input["state"]["units"]:
                if unit in input["state"]["units"][player]:
                    out['coordinator'] = player

            if len(unit.split()[0]) != 1: # if we have sth like HOL D: "void"
                assert input['results'][unit][0] == 'void', input['results'][unit]
                location = unit.split()[0]
                assert unit.split()[1] == 'D', unit.split()[1]
                # find the corresponding key in the results
                for key in input['results']:
                    if key.split()[1] == location:
                        assert input['results'][key][0] == "disband", input['results'][key]
                        input['results'][key].append(input['results'][unit][0])
                        break
            else:
                out['type'] = unit.split()[0]
                out['current_location'] = unit.split()[1]
                out['results'] = input['results'][unit]
                out['action'] = -2
                out['impact_location'] = -2
                yield(out)

In [ ]:
all_records = []
for idx, game in enumerate(games):
  for idx, phase in enumerate(game):
    row_generator = flatten_json(phase)
    assert row_generator is not None, row_generator
    for row in row_generator:
      all_records.append(row.copy())

In [ ]:
complete_df = pd.DataFrame.from_records(all_records)

In [ ]:
complete_df.loc[complete_df['coordinator'] == 'RA']

In [ ]:
complete_df['unique_unit_id'] = -1

## Temporal Binding

In [ ]:
def assign_unit_id(phase_df, source_unit_id_map, dest_unit_id_map, _id):

    # fror each row in the phase df
    for idx, row in phase_df.iterrows():

        # if current location or type of army in invalid skip the row (we only deal with valid orders)
        if row['action'] == -1 or row['type'] == 'N':
            continue
        
        # get the current location of the unit
        source_unit = row['type'] + ' ' + row['current_location']

        # if the location is not in the map, add it to the map (in other phases the same unit can be used, hence checking the condition _ dictionaries are global, have data across phases)
        if source_unit not in source_unit_id_map:
            source_unit_id_map[source_unit] = _id
            _id += 1

        # destination dict is synced with source dict after the loop, so that we can use updated info at the beginning of each assignment
        # row['unique_unit_id'] = source_unit_id_map[source_unit]
        phase_df.loc[idx,'unique_unit_id'] = source_unit_id_map[source_unit]

        if row['action'] == '-':
            result = row['results']
            if isinstance(result, list):
                if len(result) == 0:
                    dest_location = row['impact_location']
                    try:
                        dest_unit = row['type'] + ' ' + dest_location
                    except:
                        print("dest location error", row)
                        return
                    if dest_unit not in dest_unit_id_map:
                        dest_unit_id_map[dest_unit] = source_unit_id_map.pop(source_unit)
                        # source_unit_id_map[source_unit]
                        # print(source_unit_id_map.pop(source_unit))
                elif 'disband' in result:
                    source_unit_id_map.pop(source_unit)
                    
        elif row['action'] == 'R':
            result = row['results']
            if isinstance(result, list):
                if len(result) == 0:
                    dest_location = row['impact_location']
                    dest_unit = row['type'] + ' ' + dest_location
                    if dest_unit not in dest_unit_id_map:
                        dest_unit_id_map[dest_unit] = source_unit_id_map.pop(source_unit)
                elif 'disband' in result:
                    if len(result) > 1:
                        if 'void' in result:
                                print(result)
                    source_unit_id_map.pop(source_unit)

        elif row['action'] == 'D':
            result = row['results']
            assert source_unit in source_unit_id_map
            if isinstance(result, list):
                if len(result) == 0:
                    source_unit_id_map.pop(source_unit)
                elif 'void' in result:
                    print("void disband", row)
        
        elif row['action'] == 'B':
            assert source_unit in source_unit_id_map

        elif row['action'] == 'H':
            assert source_unit in source_unit_id_map

        elif row['action'] == 'S':
            assert source_unit in source_unit_id_map
        
        elif row['action'] == 'C':
            assert source_unit in source_unit_id_map

        # for added result rows
        elif row['action'] == -2:
            result = row['results']
            assert source_unit in source_unit_id_map
            if isinstance(result, list):
                if len(result) == 0:
                    source_unit_id_map.pop(source_unit)
                elif 'disband' in result and 'void' not in result:
                    source_unit_id_map.pop(source_unit)
                    # print("disband")
                elif 'disband' in result and 'void' in result:
                    # print("void disband")
                    pass
        else:
            print("invalid action", row)

    # merge the source and destination dictionaries into one
    source_unit_id_map.update(dest_unit_id_map)
    # remove the destination dict (values get updated based on old data if we don't do this)
    dest_unit_id_map = {}

    return source_unit_id_map, dest_unit_id_map, _id

In [ ]:
unique_games = complete_df["game_id"].unique()

In [ ]:
game_phase_df_list = []
for idx, game_id in enumerate(unique_games):
    print(idx, game_id)
    s_dict = {}
    d_dict = {}
    _id = 1
    game_df = complete_df.loc[complete_df["game_id"].apply(lambda x: x == game_id)]
    unique_phases = game_df['phase_id'].unique()
    for phase in unique_phases:
        condition = game_df["phase_id"].apply(lambda x: x == phase)
        phase_df = game_df.loc[condition]
        s_dict, d_dict, _id = assign_unit_id(phase_df, s_dict, d_dict, _id)
        game_phase_df_list.append(phase_df)

In [ ]:
# cdf = pd.concat(game_phase_df_list, ignore_index=True)
cdf = pd.concat(game_phase_df_list)

## Check the concatenated df

reduce to only 2 seasons per year

In [ ]:
spring_fall_phases=(cdf['phase_name'].apply(lambda x:x[0])!='W') & (cdf['phase_name'].apply(lambda x:x[-1])!='R') & (cdf['phase_name'].apply(lambda x:x[-1]) == 'M')
cdf2=cdf[spring_fall_phases].copy()
cdf2.phase_name=cdf2.phase_name.apply(lambda x: float(x[1:-1]+('.0' if x[0]=='S' else '.5')))

compute a binary winner label

In [ ]:
winner_series=pd.Series(index=cdf.game_id.unique())
for game_id in winner_series.index:
    tmax=cdf2[cdf2.game_id==game_id].phase_id.max()
    final_units=cdf2[(cdf2.game_id==game_id) & (cdf2.phase_id==tmax)].groupby('coordinator').unique_unit_id.nunique()
    winner_series[game_id]=final_units.idxmax() if len(final_units)>0 else np.nan
def dummy(row,winner_series):
    return row.coordinator==winner_series[row.game_id]
from functools import partial
part_dummy=partial(dummy,winner_series=winner_series)
cdf2['winner']=cdf2.apply(part_dummy,axis=1)

view winning stats

In [ ]:
winner_series.value_counts()

compute winner loser plot

In [ ]:
unitcounts=cdf2.groupby(["game_id","coordinator","phase_name","winner"])["unique_unit_id"].nunique()
import seaborn as sns
import pylab as pl
# winlose_data.groupby(['winner','phase_name']).mean().unstack().T.plot(legend=False,xticks=np.arange(1900,1920,5))
fig,ax=pl.subplots(1,1,figsize=(3,3))#ensemble statistics
sns.lineplot(ax=ax,
             x='phase_name',
             y='unique_unit_id',
             data=unitcounts.reset_index().drop(labels=['game_id','coordinator'],axis=1),
             hue=r'winner',
             ci='sd',
             palette=sns.color_palette("colorblind", 2))
h,l=ax.get_legend_handles_labels()
ax.legend(handles=h[1:][::-1],labels=['winners','losers'],frameon=False)
ax.grid()
ax.set_yticks(np.arange(0,20,2))
ax.set_yticklabels([0,'',4,'',8,'',12,'',16,''])
ax.set_xticks(np.arange(1901,1919,4))
ax.set_ylabel('number of units')
ax.set_xlabel('year')
ax.set_ylim(0,18)
ax.set_xlim(1901,1918)
# fig.savefig('winners_and_losers.pdf',format='pdf',dpi=300,bbox_inches='tight')

In [ ]:
complete_df.loc[complete_df["game_id"] == 'uXFQ2zgI-DUrgwlS']['phase_id'].unique()

In [ ]:
cdf.loc[cdf['coordinator'] == 'RA'][:40].sort_values(by=['phase_id', 'unique_unit_id'])

In [ ]:
cdf.loc[cdf['game_id'] == 'lVp6PZxk3Jpufc9Z']

In [ ]:
unique_games

In [ ]:
spring_fall_phases=(cdf['phase_name'].apply(lambda x:x[0])!='W') & (cdf['phase_name'].apply(lambda x:x[-1])!='R') & (cdf['phase_name'].apply(lambda x:x[-1]) == 'M')
cdf[spring_fall_phases].groupby(["game_id","coordinator","phase_name"])["unique_unit_id"].nunique().unstack().T.plot()

In [ ]:
cdf[spring_fall_phases].groupby(["game_id","coordinator","phase_name"])["unique_unit_id"].nunique().unstack().head(14)
cdf.groupby(["game_id","coordinator","phase_id"])["unique_unit_id"].nunique().unstack().head(8)

In [ ]:
cnd=(cdf['phase_name'].apply(lambda x:x[0])!='W') & (cdf['phase_name'].apply(lambda x:x[-1])!='R') & (cdf['game_id'].apply(lambda x: x == '0yv59hl6e5Ensb4M') )
cdf[cnd].groupby(["game_id","coordinator","phase_id"])["unique_unit_id"].nunique().unstack().head(7)

In [ ]:
cdf[spring_fall_phases].loc[cdf["game_id"] == "0yv59hl6e5Ensb4M"]

## Save a single game to JSON 

In [ ]:
from pprint import pprint
import json

In [ ]:
game_id = "uXFQ2zgI-DUrgwlS"
for game in games_jsons:
    if game['id'] == game_id:
        json.dump(game, open(f'{game_id}.json', 'w'))

# Calculate different triples

### Manual exploration

In [ ]:
# for a single game:
game_id = "lVp6PZxk3Jpufc9Z"
game_df = cdf.loc[cdf["game_id"] == game_id]

In [ ]:
game_df

In [ ]:
game_df["unique_unit_id"].nunique()

In [ ]:
game_df["unique_unit_id"].max()

In [ ]:
game_df.loc[game_df["unique_unit_id"].idxmax()]

In [ ]:
players = game_df["coordinator"].unique()
assert len(players) == 7

In [ ]:
player_units = {}
for player in players:
    player_units[player] = game_df.loc[game_df["coordinator"] == player]["unique_unit_id"].unique()

In [ ]:
#https://stackoverflow.com/questions/45655936/how-to-test-all-items-of-a-list-are-disjoint
# make sure some units are handed over to other players

import itertools
def all_disjoint(iterables):
    merged = itertools.chain(*iterables)
    total = list(merged)
    total.sort()
    print(total)
    print(set(total))
    return len(total) == len(set(total))

In [ ]:
all_units = list(player_units.values())

In [ ]:
all_disjoint(all_units)

In [ ]:
player_units

In [ ]:
unique_two_players = set(itertools.permutations(player_units.keys(), 2))

In [ ]:
len(unique_two_players)

In [ ]:
from collections import namedtuple
Triple = namedtuple('Triple', ['triple', 'player_i', 'player_j'])

In [ ]:
triples = []
for player_i, player_j in unique_two_players:
    if player_i== "TURKEY" and player_j == "AUSTRIA":
        print(player_i, player_j)
        p_units = list(itertools.combinations(player_units[player_i], 2))
        o_unit = list(itertools.combinations(player_units[player_j], 1))
        # get all 3-tuples unique combinations
        p_o_product = list(itertools.product(p_units, o_unit))
        p_o_product = [(a, b, c) for (a,b), (c,) in p_o_product]
        triple = [Triple(t, player_i, player_j) for t in p_o_product]
        triples.append(triple)
        # first tuple in the product belongs to player_i and the other belongs to player_j
        break

In [ ]:
len(triples)

In [ ]:
Triple(p_1_name='TURKEY', p1_val=(1,2), p2_name='AUSTRIA', p2_val=3)

In [ ]:
p_units = list(itertools.combinations(player_units["TURKEY"], 2))
o_unit = list(itertools.combinations(player_units["AUSTRIA"], 1))

In [ ]:
p_units

In [ ]:
result = itertools.product(p_units, ["TURKEY"])

In [ ]:
list(result)

In [ ]:
for i, _tuple in enumerate(all_tuples):
    all_tuples[i] = _tuple[0] + _tuple[1]

In [ ]:
len(all_tuples)

In [ ]:
all_tuples

In [ ]:
# p_units = set(itertools.combinations([1,2,3], 2))
# o_unit = set(itertools.combinations([4,5], 1))

# # get all 3-tuples unique combinations
# all_tuples = list(itertools.product(p_units, o_unit))

In [ ]:
#https://www.geeksforgeeks.org/python-remove-duplicate-tuples-from-list-of-tuples/
def removeDuplicates(lst):
    return list(set([i for i in lst]))

In [ ]:
all_unique_tuples = removeDuplicates(all_tuples)

In [ ]:
len(all_unique_tuples)

In [ ]:
for triple in all_unique_tuples:
    print(triple)
    condition = game_df["unique_unit_id"].apply(lambda x: x in triple)
    presence = game_df[condition]
    break

In [ ]:
presence.sort_values(['unique_unit_id'])

In [ ]:
presence[presence["unique_unit_id"] == 22]

In [ ]:
presence[presence["unique_unit_id"] == 22]["phase_name"].value_counts()

In [ ]:
phases = presence["phase_name"].value_counts() == 2

In [ ]:
phases.to_dict()

In [ ]:
eligible_phases = presence.loc[presence["phase_name"].apply(lambda x: x in phases and phases[x])]

In [ ]:
eligible_phases

In [ ]:
old_ids = eligible_phases["phase_id"].unique()

In [ ]:
old_ids = old_ids.tolist()

In [ ]:
new_ids = np.arange(1, len(old_ids)+1)

In [ ]:
eligible_phases

In [ ]:
phase_ids = presence.loc[presence["phase_name"].apply(lambda x: x in phases and phases[x])].phase_id.unique()

In [ ]:
phase_ids.max()

In [ ]:
phase_ids.min()

In [ ]:
triple

### Sanity check that the code works

In [ ]:
for game_id in unique_games:
    game_df = cdf.loc[cdf["game_id"] == game_id]
    assert game_df["unique_unit_id"].nunique() == game_df["unique_unit_id"].max(), game_id

In [ ]:
triples = []
for game_id in unique_games:
    game_df = cdf.loc[cdf["game_id"] == game_id]

# Loading Deepmind's Diplomacy trajectory

In [ ]:
action_outputs = np.load(file='../data/actions_outputs.npz', allow_pickle=True)

In [ ]:
action_outputs[1][0]

In [ ]:
pd.DataFrame.from_dict(action_outputs[0][0]).corr(method='pearson')

In [ ]:
legal_actions = np.load(file='../data/legal_actions.npz', allow_pickle=True)

In [ ]:
len(legal_actions)

In [ ]:
legal_actions[0].shape

In [ ]:
observations = np.load(file='../data/observations.npz', allow_pickle=True)

In [ ]:
len(observations)

In [ ]:
observations[1]

In [ ]:
step_outputs = np.load(file='../data/step_outputs.npz', allow_pickle=True)

In [ ]:
len(step_outputs)

In [ ]:
step_outputs[0]